In [1]:
import xarray as xr
import numpy as np
import pandas as pd, xarray as xr
import feather, h5py, sys, pickle

In [3]:
path = '../annual_emissions/na_surfaces_01.mat'
arrays = {}
with h5py.File(path, 'r') as file:
	for k,v in file.items():
		arrays[k] = np.array(v)

In [4]:
areas = arrays['areas'] # these are the surfaces along one longitude band, need to repeat
# the values for all longitudes
surfaces = np.ones((400,900))
surfaces *= np.transpose(areas)

In [5]:
surfaces.shape

(400, 900)

In [6]:
lats = np.arange(20.05, 60, 0.1)
lons = np.arange(-139.9, -49.95, 0.1)
surfaces = xr.DataArray(data=surfaces, coords=[lats,lons], dims=['lat', 'lon'])

In [7]:
surfaces.isel(lat=-1)

<xarray.DataArray (lon: 900)>
array([61821550.710798, 61821550.710798, 61821550.710798, ..., 61821550.710798,
       61821550.710798, 61821550.710798])
Coordinates:
    lat      float64 59.95
  * lon      (lon) float64 -139.9 -139.8 -139.7 -139.6 ... -50.2 -50.1 -50.0

In [9]:
# Get power model output and merge with power plant characteristics
gen = feather.read_dataframe(f'./outputs/gen_no-nuclear_modified-all-generators.feather')
carac = pd.read_csv(f'./good_model_inputs/inputs_gen_no-nuclear_all-generators_20k-new_name.csv')

# Clean columns name
carac = carac.drop('Unnamed: 0', axis=1)

# Merge
opt_out = pd.concat((carac,gen), axis=1)

# Add the missing final hour
opt_out['2017_365_23'] = opt_out['2017_365_22'].copy()

/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
gen

,2017_1_0,2017_1_1,2017_1_2,2017_1_3,2017_1_4,2017_1_5,2017_1_6,2017_1_7,2017_1_8,2017_1_9,...,2017_365_13,2017_365_14,2017_365_15,2017_365_16,2017_365_17,2017_365_18,2017_365_19,2017_365_20,2017_365_21,2017_365_22
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
opt_out['2017_1_0']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
16850    0.0
16851    0.0
16852    0.0
16853    0.0
16854    0.0
Name: 2017_1_0, Length: 16855, dtype: float64

In [11]:
# Check for missing longitudes
opt_out[opt_out.LON == 0]

,RegionName,FuelType,FuelCostTotal,PLCO2RTA,ORISCode,DIVISIONCE,UNITID,Capacity,OBJECTID,UniqueID,...,2017_365_14,2017_365_15,2017_365_16,2017_365_17,2017_365_18,2017_365_19,2017_365_20,2017_365_21,2017_365_22,2017_365_23


In [12]:
# Compute grid indices of all generators on NEI grid
startlat = 20.05
startlon = -139.9
step = 0.1
opt_out['idxLat'] = np.floor((opt_out['LAT'].values - startlat)/step)
opt_out['idxLon'] = np.floor((opt_out['LON'].values - startlon)/step)

#Check that all values are within grid limits
if (opt_out['idxLat'] < 0).sum() + (opt_out['idxLat'] > 400).sum() +\
	(opt_out['idxLon'] < 0).sum() + (opt_out['idxLon'] > 900).sum() != 0:
	raise ValueError('Some generators are outside grid limits')

In [13]:
no[[f'2017_{day}_{hour}' for day in range(1,366) for hour in range(24)]] *= 2

NameError: name 'no' is not defined

In [ ]:
# Process NO emissions
no = pd.DataFrame(data=opt_out['egrid_nox'].values.reshape(len(opt_out.index),1) * opt_out[[list(opt_out)[i] for i in range(32,8792)]] / 3600)
# no is now in lb/s
no['idxLat'] = opt_out['idxLat'].astype(int)
no['idxLon'] = opt_out['idxLon'].astype(int)
no['StateName'] = opt_out['StateName']
mult = 0.45359 * 0.8544304 # 0.45 kg / lb * NO/NOx as estimated from NEI2011 inventory
no[[f'2017_{day}_{hour}' for day in range(1,366) for hour in range(24)]] *= mult
# no is now in kg/s

# Process NO2 emissions
#no2 = pd.DataFrame(data=opt_out['PLNOXRTA'].values.reshape(len(opt_out.index),1) * opt_out[[list(opt_out)[i] for i in range(32,8792)]] / 3600)
## no is now in lb/s
#no2['idxLat'] = opt_out['idxLat'].astype(int)
#no2['idxLon'] = opt_out['idxLon'].astype(int)
#no2['StateName'] = opt_out['StateName']
#no2 *= 0.45359 # 0.45 kg / lb
#no2 *= 0.1455696 # NO2/NO as estimated from NEI2011 inventory
## no2 is now in kg/s
#
#so2 = pd.DataFrame(data=opt_out['PLSO2RTA'].values.reshape(len(opt_out.index),1) * opt_out[[list(opt_out)[i] for i in range(32,8792)]] / 3600)
## so2 is now in lb/s
#so2['idxLat'] = opt_out['idxLat'].astype(int)
#so2['idxLon'] = opt_out['idxLon'].astype(int)
#so2['StateName'] = opt_out['StateName']
#so2 *= 0.45359 # 0.45 kg / lb
# so2 is now in kg/s

In [ ]:
# Integrate surface processing and create Dataset
path = './na_surfaces_01.mat'
arrays = {}
with h5py.File(path, 'r') as file:
	for k,v in file.items():
		arrays[k] = np.array(v)
		
areas = arrays['areas'] # these are the surfaces along one longitude band, need to repeat
# the values for all longitudes
surfaces = np.ones((400,900))
surfaces *= np.transpose(areas)

In [ ]:
# Function that returns offset from UTC based on lat/lon
from timezonefinder import TimezoneFinder
from pytz import timezone
import pytz
from datetime import datetime

utc = pytz.utc
tf = TimezoneFinder(in_memory=True)

def offset(lat, lon):
    """
    returns a location's time zone offset from UTC in minutes.
    """
    today = datetime.now()
    tz_target = timezone(tf.certain_timezone_at(lat=lat, lng=lon))
    # ATTENTION: tz_target could be None! handle error case

    # today_target = tz_target.localize(today)
    # today_utc = utc.localize(today)
    # offset = today_utc - today_target
    offset = tz_target.utcoffset(today)

    # if `today` is in summer time while the target isn't, you may want to substract the DST
    offset -= tz_target.dst(today)
    return int(offset.total_seconds() / 3600)

In [ ]:
no.head()

In [ ]:
idx=5
myno = np.zeros((8760,400,900))
ofs = -offset(opt_out.loc[idx,'LAT'],opt_out.loc[idx,'LON'])

myno[:, no.loc[idx, 'idxLat'], no.loc[idx, 'idxLon']] = np.roll(no.iloc[idx,:8760],ofs).astype(float)

In [ ]:
# Grid emissions
# Need to shift the hours to account for time zone
myno = np.zeros((8760,400,900))
#myno2 = np.zeros((8760,400,900))
#my2o2 = np.zeros((8760,400,900))

# Shift the hours to have everything at UTC time for GEOS-Chem
for idx in no.index:
    ofs = -offset(opt_out.loc[idx,'LAT'],opt_out.loc[idx,'LON'])
	myno[:, no.loc[idx, 'idxLat'], no.loc[idx, 'idxLon']] += np.roll(no.iloc[idx,:8760],ofs)
    #myno2[:, myno2.loc[idx, 'idxLat'], myno2.loc[idx, 'idxLon']] += np.roll(myno2.loc[idx,:8760],ofs)
    #myso2[:, myso2.loc[idx, 'idxLat'], myso2.loc[idx, 'idxLon']] += np.roll(myso2.loc[idx,:8760],ofs)
myno = myno/surfaces # nox in kg/m2/s
#myno2 = myno2/surfaces
#myso2 = myso2/surfaces

In [ ]:
# Load mask of us states
with open('mask_us_neigrid.pkl', 'rb') as f:
    mask = pickle.load(f)

In [ ]:
# Add power plant emissions outside of the US
for day in range(1,366):
    # Load NEI values for that day
    mth = pd.to_datetime(f'2017+{day}', format='%Y+%j').strftime(format='%m')
    dat = pd.to_datetime(f'2017+{day}', format='%Y+%j').strftime(format='%Y%m%d')
    nei1 = xr.open_dataset(f'/net/fs10/d1/seastham/GCData/ExtData/HEMCO/NEI2011/v2015-03/{mth}/NEI11_0.1x0.1_{dat}_egu.nc')
    nei2 = xr.open_dataset(f'/net/fs10/d1/seastham/GCData/ExtData/HEMCO/NEI2011/v2015-03/{mth}/NEI11_0.1x0.1_{dat}_egupk.nc')
    for t in range(24):
        # where there are no US states, copy the values ffrom NEI
        myno[(day-1)*24+t,:,:][mask == -1] = nei1['NO'].isel(time=t, lev=0).values[mask == -1] +\
                                                nei2['NO'].isel(time=t, lev=0).values[mask == -1]
        myno2[(day-1)*24+t,:,:][mask == -1] = nei1['NO2'].isel(time=t, lev=0).values[mask == -1] +\
                                                nei2['NO2'].isel(time=t, lev=0).values[mask == -1]
        myso2[(day-1)*24+t,:,:][mask == -1] = nei1['SO2'].isel(time=t, lev=0).values[mask == -1] +\
                                                nei2['SO2'].isel(time=t, lev=0).values[mask == -1]

In [ ]:
# Initialize dataset dimensions and create the dataarrays
lat = np.arange(20.05, 60, 0.1)
lon = np.arange(-139.9, -49.95, 0.1)
time = pd.date_range(start='2017-01-01', freq='H', periods=8759)
NO = xr.DataArray(myno, coords=[time, lat, lon], dims=['time', 'lat', 'lon'], \
	attrs={'long_name': 'NO power plant emissions',\
		   'units': 'kg/m2/s'})
NO2 = xr.DataArray(myno2, coords=[time, lat, lon], dims=['time', 'lat', 'lon'], \
	attrs={'long_name': 'NO2 power plant emissions',\
		   'units': 'kg/m2/s'})
SO2 = xr.DataArray(myso2, coords=[time, lat, lon], dims=['time', 'lat', 'lon'], \
	attrs={'long_name': 'SO2 power plant emissions',\
		   'units': 'kg/m2/s'})

In [ ]:
lat = np.arange(20.05, 60, 0.1)
lon = np.arange(-139.95, -49.95, 0.1)

In [ ]:
lat

In [ ]:
datetime.today().strftime(format='%Y/%m/%d')

In [ ]:
########################################### Push final output ###########################################
# Build new netCDF file
f_out = f'/net/d13/data/gchossie/EVChina_GEOS_Chem/Annual_emissions/inventory_recent{suffix}.nc'

if os.path.isfile(f_out):
	print('Clobbering ' + f_out)
	os.remove(f_out)
nc_out = netCDF4.Dataset(f_out,'w',format='NETCDF4')
tdy = datetime.datetime.now().strftime(format='%Y-%m-%d')

# Global attributes
nc_out.Title = 'Hourly CH power plant inventory'
nc_out.Conventions = 'COARDS'
nc_out.History = f'Created {tdy}',
nc_out.Contact = 'gchossie@mit.edu'
dim_dict = {'time': {'units': 'hours since 2017-01-01 00:00:00',
					 'dtype': 'i4',
					 'data':  list(range(8760)),
					 'other': {'calendar': 'gregorian'}},
			'lat':  {'units': 'degrees_north',
					 'dtype': 'f8',
					 'data':  lat,
					 'other': {'long_name': 'latitude'}},
			'lon':  {'units': 'degrees_east',
					 'dtype': 'f8',
					 'data':  lon,
					 'other': {'long_name': 'longitude'}}}

for dim_name, dim_data in dim_dict.items():
	nc_out.createDimension(dim_name,len(dim_data['data']))
	var_temp = nc_out.createVariable(dim_name,dim_data['dtype'],(dim_name))
	var_temp[:] = dim_data['data'][:],
	var_temp.units = dim_data['units']
	for att_name, att_val in dim_data['other'].items():
		var_temp.setncattr(att_name,att_val)

chunks = [1,len(lat),len(lon)]
var_list = ['NO','SO2','PM25']
trad = {'NO': myno, 'SO2': myso2, 'PM25': mypm25}

for var in var_list:
    nc_var = nc_out.createVariable(var,'f8',(('time','lat','lon')),
									 chunksizes=chunks,complevel=1,zlib=True)
    nc_var.units = 'kg/s/m2'
    nc_var.long_name = f'{var} power plant emissions from my optimization mode'
    src_data = trad[var]
    
    for i_time in range(8760):
        var_data = src_data[i_time,:,:]
        nc_var[i_time,:,:] = var_data.copy()

nc_out.close()